In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split


In [2]:
dataset_path = r'C:\\Users\\Dell\\Desktop\\MANI\\ndp\\UTKFace'


In [3]:
def load_images_and_ages(dataset_path):
    images = []
    ages = []

    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg"):
            parts = filename.split('_')

            if len(parts) < 4:
                continue  # Skip files that don't follow the expected naming pattern

            try:
                age = int(parts[0])  # Extract age from the filename (first part)
                img_path = os.path.join(dataset_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (48, 48))  # Resize to 48x48
                img = img.astype('float32') / 255.0  # Normalize the image

                images.append(img)
                ages.append(age)
            except ValueError:
                continue  # Skip files with invalid age labels

    images = np.array(images)
    ages = np.array(ages)
    return images, ages


In [4]:
images, ages = load_images_and_ages(dataset_path)

In [5]:
max_age = ages.max()
ages_normalized = ages / max_age


In [6]:
X_train, X_val, y_train, y_val = train_test_split(images, ages_normalized, test_size=0.2, random_state=42)


In [7]:
def build_age_prediction_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Single output neuron for regression
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

In [8]:
age_model = build_age_prediction_model()


C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
age_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 26s 40ms/step - loss: 0.0406 - mae: 0.1396 - val_loss: 0.0130 - val_mae: 0.0878
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - loss: 0.0127 - mae: 0.0850 - val_loss: 0.0108 - val_mae: 0.0761
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - loss: 0.0098 - mae: 0.0744 - val_loss: 0.0091 - val_mae: 0.0726
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - loss: 0.0085 - mae: 0.0678 - val_loss: 0.0084 - val_mae: 0.0680
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - loss: 0.0074 - mae: 0.0643 - val_loss: 0.0079 - val_mae: 0.0643
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 23s 39ms/step - loss: 0.0068 - mae: 0.0613 - val_loss: 0.0080 - val_mae: 0.0654
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - loss: 0.0063 - mae: 0.0589 - val_loss: 0.0080 - val_mae: 0.0645
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - loss: 0.0059 - mae: 0.0572 - val_loss: 0.0075 - val_mae: 0.0635
Epoch 9/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 

In [10]:
loss, mae = age_model.evaluate(X_val, y_val)
print(f"Validation Mean Absolute Error: {mae * max_age:.2f} years")


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0076 - mae: 0.0633
Validation Mean Absolute Error: 7.38 years


In [11]:
age_model.save('age_prediction_model.h5')
